In [223]:
import pandas as pd
import json
import re
import numpy as np
import ast
from pathlib import Path  

In [224]:
main_path = "/home/rvissche/Nextcloud/What-If/what-if-data-donation/what-if-data-donation/json_structure_donations/processed_json_structure_donations/"

In [ ]:
# Define the data types
data_types = ['string', 'array', 'number', 'boolean', 'object']

def process_json_2(row):
    if row['json_2'] in data_types:
        row['data_type'] = row['json_2']
        row['json_2'] = np.nan
    return row

def process_json_3(row):
    if row['json_3'] in data_types:
        row['data_type'] = row['json_3']
        row['json_3'] = np.nan
    return row

def process_json_4(row):
    if row['json_4'] in data_types:
        row['data_type'] = row['json_4']
        row['json_4'] = np.nan
    return row

def process_json_5(row):
    if row['json_5'] in data_types:
        row['data_type'] = row['json_5']
        row['json_5'] = np.nan
    return row



def get_json_3(row):
        level1 = row['value'].get(row['json_1'], None)
        if isinstance(level1, dict):
            return level1.get(row['json_2'], None)
        return None

def get_json_4(row):
        level1 = row['value'].get(row['json_1'], None)
        if isinstance(level1, dict):
            level2= level1.get(row['json_2'], None)
        if isinstance(level2, dict):
            return level2.get(row['json_3'], None)
        return None

def get_json_5(row):
        level1 = row['value'].get(row['json_1'], None)
        if isinstance(level1, dict):
            level2= level1.get(row['json_2'], None)
        if isinstance(level2, dict):
            level3= level2.get(row['json_3'], None)
        if isinstance(level3, dict):
            return level3.get(row['json_4'], None)
        return None



In [ ]:

def structure_donations(data):

    data = Path(data)  
    #file_name = data.name  # Extracts "data.json"
    file_name = Path(data).stem 

    # Load JSON file
    with open(data, 'r') as f:
        data = json.load(f)

    # Flatten JSON (handling nested structures)
    df = pd.json_normalize(data, max_level=0)

    # Delete user specific informations
    #df.columns = df.columns.str.replace(r'^[^/]+/', '', regex=True)

    # Extract column names
    cols = df.columns.tolist()


    # From wide to long df
    df = pd.melt(df, value_vars= cols)


    # Unlist the value column (where the JSON info is stored)
    for index, row in df.iterrows():
        if isinstance(row['value'], list):
            df.at[index, 'value'] = row['value'][0]


    # Create an emtpy column for the level 1 JSON 
    df['json_1'] = ''

    # Extract the level 1 keys
    for index, row in df.iterrows():
        df.at[index, 'json_1'] = list(row['value'].keys())

    # Take the level 1 keys stored in a list and store them in individual rows
    df = df.explode('json_1')

    # For the level 1 keys stored in json_1 extract the level 2 keys and store in json_2
    df['json_2'] = df.apply(lambda row: [row['value'].get(row['json_1'], None)], axis=1)

    # Unlist and store in individual rows
    df['json_2'] = df['json_2'].apply(lambda x: x[0] if isinstance(x, list) else x)
    df['json_2'] = df['json_2'].apply(lambda x: x[0] if isinstance(x, list) else x)
    df = df.explode('json_2')
    # If a data type is stored in json_2 (data types aren't keys) replace with Na and store in the column data_type
    df = df.apply(process_json_2, axis=1)

    #print(df.head())

    
    #JSON 3
    df['json_3'] = df.apply(get_json_3, axis=1)
    # Unlist and store in individual rows
    df['json_3'] = df['json_3'].apply(lambda x: x[0] if isinstance(x, list) else x)
    df = df.explode('json_3')
    df = df.apply(process_json_3, axis=1)
    
    #JSON 4
    df['json_4'] = df.apply(get_json_4, axis=1)
    # Unlist and store in individual rows
    df['json_4'] = df['json_4'].apply(lambda x: x[0] if isinstance(x, list) else x)
    df = df.explode('json_4')
    df = df.apply(process_json_4, axis=1)

    #JSON 5
    df['json_5'] = df.apply(get_json_5, axis=1)
    # Unlist and store in individual rows
    df['json_5'] = df['json_5'].apply(lambda x: x[0] if isinstance(x, list) else x)
    df = df.explode('json_5')
    df = df.apply(process_json_5, axis=1)

     #JSON 6
    df['json_6'] = df.apply(get_json_6, axis=1)
    # Unlist and store in individual rows
    df['json_6'] = df['json_6'].apply(lambda x: x[0] if isinstance(x, list) else x)
    df = df.explode('json_6')
    df = df.apply(process_json_6, axis=1)
    

    
    # Reorder the columns in the df
    df = df.loc[:, ['variable', 'value', 'json_1', 'json_2', 'json_3', 'json_4' , 'json_5', 'data_type']]
   

    df.reset_index(drop=True, inplace=True)
    
    dfs = {}
    dfs[file_name] = df

    df.to_csv(f"{main_path}TikTok/Output/Output_" + file_name + '.csv', index=False)
    




In [227]:
input_directory = Path(f'{main_path}TikTok/Input')  
print(input_directory)

/home/rvissche/Nextcloud/What-If/what-if-data-donation/what-if-data-donation/json_structure_donations/processed_json_structure_donations/TikTok/Input


In [228]:


for file in input_directory.iterdir():  
    if file.is_file():  
        print(file)
        structure_donations(file)

/home/rvissche/Nextcloud/What-If/what-if-data-donation/what-if-data-donation/json_structure_donations/processed_json_structure_donations/TikTok/Input/TT_json_structure_MvdV.json


UnboundLocalError: cannot access local variable 'level4' where it is not associated with a value

In [ ]:
# Path to the folder containing CSV files
output_path = f"{main_path}TikTok/Output"

# Get a list of all CSV files in the folder
csv_files = list(Path(output_path).glob("*.csv"))

# Load all CSVs into a list of DataFrames
dfs = [pd.read_csv(file) for file in csv_files]


common_columns = ['variable', 'value', 'json_name', 'json_1', 'json_2', 'json_3', 'json_4', 'json_5','data_type']

merged_df = dfs[0]  # Start with the first DataFrame
for df in dfs[1:]:  # Merge with the rest
    merged_df = merged_df.merge(df, on=common_columns, how="outer")

"""
# Filter where col1 contains 'messages', then drop duplicates based on col2
df_filtered = merged_df[merged_df["path_1"] == "messages"].drop_duplicates(subset="path_2")


# Append rows where col1 does not contain 'messages'
df_final = pd.concat([df_filtered, merged_df[merged_df["path_1"] != "messages"]], ignore_index=True)
"""

# Save the final merged DataFrame
merged_df.to_csv(f"{main_path}TikTok/Final/Merged_structures_TT.csv", index=False)